In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
!echo $CUDA_VISIBLE_DEVICES

In [ ]:
!nvidia-smi

In [ ]:
import sys
sys.path.insert(0, "../")

import os
os.chdir("../")

In [ ]:
problems = ["dw4", "lj13", "qm9"]
problem = problems[0]
if problem == "dw4":
    dim = 2
    n_nodes = 4
    from examples.dw4_fab import *
    import target.double_well as dw4
    target_log_prob_fn = dw4.log_prob_fn
    load_dataset = load_dataset_original
# elif problem == "lj13":
#     dim = 3
#     n_nodes = 13
#     from examples.lj13 import *
#     import target.leonard_jones as lj13
#     target_log_prob_fn = lj13.log_prob_fn
# else:
#     dim = 3
#     n_nodes = 19
#     from examples.qm9 import *
#     target_log_prob_fn = None

In [ ]:
import yaml
import jax
import matplotlib.pyplot as plt

In [ ]:
USE_64_BIT = False
if USE_64_BIT:
    from jax.config import config
    config.update("jax_enable_x64", True)

In [ ]:
cfg = DictConfig(yaml.safe_load(open(f"examples/config/{problem}_fab.yaml")))
cfg

# Train

In [ ]:
cfg.training.save = False
cfg.logger = DictConfig({"list_logger": None})
cfg.flow.act_norm = False

In [ ]:
# Flow setup
cfg.flow.n_aug = 1
cfg.flow.type = 'spherical'
cfg.flow.n_layers = 8

# AIS setup
cfg.fab.transition_operator.metropolis.init_step_size = 0.2

In [ ]:
# Training setup.
cfg.training.optimizer.use_schedule = False
cfg.training.optimizer.init_lr = 3e-4
cfg.training.optimizer.max_global_norm = 1.0
cfg.training.n_epoch = int(10000)
cfg.training.batch_size = 100

In [ ]:
experiment_config = create_train_config(cfg, dim=dim, n_nodes=n_nodes, target_log_p_x_fn=log_prob_fn,
                                        load_dataset=load_dataset)

logger, state = train(experiment_config)

# 